In [1]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
# executeme

#@time begin
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

using LinearAlgebra

#end

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [125]:
Nx = 8; Ny = 8; N=Nx*Ny; q = Nx; N_cut = q
PN = 4
U = 2

2

In [126]:
# executeme

function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

In [127]:
basis = NLevelBasis(N)

NLevel(N=64)

In [128]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=64x64)
  basis: NLevel(N=64)sparse([2, 8, 9, 57, 1, 3, 10, 58, 2, 4  …  61, 63, 7, 55, 62, 64, 8, 56, 57, 63], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 38, 38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 44, 44, 44, 44, 45, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 47, 48, 48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 50, 51, 51, 51, 51, 52, 52, 52, 52, 53, 53, 53, 53, 54, 54, 54, 54, 55, 55, 55, 55, 56, 56, 56, 56, 57, 57, 57, 57,

In [129]:
eigenenergies(dense(H1))

64-element Vector{Float64}:
 -3.2906575520321466
 -3.290657552032146
 -3.2906575520321453
 -3.2906575520321453
 -3.2906575520321453
 -3.290657552032145
 -3.290657552032145
 -3.2906575520321435
 -2.000000000000001
 -2.000000000000001
 -2.0
 -2.0
 -2.0
  ⋮
  2.0000000000000018
  2.0000000000000018
  2.0000000000000018
  2.000000000000003
  3.290657552032139
  3.29065755203214
  3.2906575520321404
  3.2906575520321417
  3.290657552032145
  3.290657552032146
  3.2906575520321466
  3.2906575520321466

In [130]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [131]:
# executeme

function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [132]:
sub_states = get_sub_states(H1, N_cut);

In [133]:
# executeme

function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [134]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [135]:
# executeme

function Subspace_Op(SP_Op, P, Pt)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [136]:
H1_sub = Subspace_Op(H1, P, Pt)

Operator(dim=8x8)
  basis: Subspace(superbasis=NLevel(N=64), states:8)
 -3.29066+2.0e-17im   4.0e-17+5.0e-17im  …  -1.7e-16+6.0e-17im
  8.0e-17-1.7e-16im  -3.29066-0.0im          3.3e-16+1.7e-16im
     -0.0+1.0e-17im  -2.2e-16+4.0e-17im     -6.0e-17-3.3e-16im
 -1.0e-16+8.0e-17im  -3.4e-16+4.0e-17im          0.0+1.1e-16im
 -1.3e-16-5.0e-17im  -3.0e-17+0.0im         -8.0e-17+1.0e-17im
  8.0e-17+8.0e-17im  -4.0e-17-1.0e-17im  …   6.0e-17+0.0im
 -8.0e-17-6.0e-17im   5.0e-17+9.0e-17im      9.0e-17-5.0e-17im
 -4.3e-16+2.0e-17im   4.0e-17-2.0e-17im     -3.29066+1.0e-17im

In [137]:
# executeme

function get_num_sub_list(basis,P,Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(basis, m, m)
        NMP = Subspace_Op(NM, P,Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [138]:
num_sub_list = get_num_sub_list(basis,P,Pt);

In [139]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:8), states:330)

In [140]:
# executeme

function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

get_MB_Op (generic function with 1 method)

In [141]:
H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub);

In [142]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [143]:
# executeme

function get_num_mb_list(basis_mb, b_sub, num_sub_list)
    num_mb_list = []
    for m in 1:N
        NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [144]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

In [145]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U);

In [146]:
#@btime begin
@time begin
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)
end

 77.154504 seconds (1.78 G allocations: 172.833 GiB, 21.03% gc time)


Operator(dim=330x330)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:8), states:330)
     0.4596+0.0im         …          0.0+0.0im
  0.0727069-0.0031256im              0.0+0.0im
  0.0165096-0.0364714im              0.0+0.0im
 -0.0950013-0.00469991im             0.0+0.0im
 -0.0416055-0.00977385im             0.0+0.0im
  0.0423214-0.0568441im   …          0.0+0.0im
  0.0132951-0.00821458im             0.0+0.0im
  0.0532268+0.0429626im              0.0+0.0im
  -0.008194+0.0302229im              0.0+0.0im
 -0.0346365+0.00496953im             0.0+0.0im
 -0.0177489-0.0206397im   …          0.0+0.0im
 -0.0238066-0.018637im               0.0+0.0im
  0.0151804-0.0330728im              0.0+0.0im
           ⋮              ⋱  
        0.0+0.0im                    0.0+0.0im
        0.0+0.0im                    0.0+0.0im
        0.0+0.0im         …  -0.00114576+0.0189515im
        0.0+0.0im                    0.0+0.0im
        0.0+0.0im                    0.0+0.0im
        

In [147]:
P1 = P.data
P1t = Pt.data;

In [148]:
@time  @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n];

  0.054616 seconds (1.86 M allocations: 56.849 MiB, 13.25% gc time)


In [149]:
@time b2_sub = b_sub ⊗ b_sub

  0.000009 seconds (2 allocations: 176 bytes)


[Subspace(superbasis=NLevel(N=64), states:8) ⊗ Subspace(superbasis=NLevel(N=64), states:8)]

In [150]:
@time P4re = reshape(P4, N_cut^2, N_cut^2);

  0.000016 seconds (2 allocations: 96 bytes)


In [151]:
@time Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re);

  0.000041 seconds (9 allocations: 160.938 KiB)


In [152]:
# executeme

function get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, N_cut^2, N_cut^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    #Vint_bsub2 = Operator(b2_sub,U/2*P4re)

    return Vint_bsub2
end

get_Vint_bsub2_einsum (generic function with 1 method)

In [153]:
@time begin
Vint_bsub2 = get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
end;

  0.065670 seconds (70.70 k allocations: 5.019 MiB, 99.00% compilation time)


In [154]:
@time begin
Vint_mb_einsum = manybodyoperator(basis_mb, Vint_bsub2)
end;

 30.426228 seconds (892.11 M allocations: 106.350 GiB, 13.94% gc time)


In [155]:
function manybodyoperator_2_Hubbard(basis::ManyBodyBasis, op::SparseOpType)
    N = length(basis)
    S = length(basis.onebodybasis)
    result = SparseOperator(basis)
    occupations = basis.occupations
    rows = QuantumOpticsBase.rowvals(op.data)
    values = QuantumOpticsBase.nonzeros(op.data)
    @inbounds for column=1:S^2, j in QuantumOpticsBase.nzrange(op.data, column)
        row = rows[j]
        value = values[j]
        index = Tuple(CartesianIndices((S, S, S, S))[(column-1)*S^2 + row])
        for m=1:N, n=1:N
            # println("row:", row, " column:"column, ind_left)
            C = QuantumOpticsBase.coefficient(occupations[m], occupations[n], index[1:2], index[3:4])
            if C!=0.
                result.data[m,n] += C*value
            end
        end
    end
    return result
end

manybodyoperator_2_Hubbard (generic function with 1 method)

In [156]:
@time Vint_mb_Hubbard = manybodyoperator_2_Hubbard(basis_mb, Vint_bsub2);

 31.367981 seconds (892.14 M allocations: 106.352 GiB, 15.09% gc time, 0.14% compilation time)


In [157]:
isapprox( Vint_mb_einsum, Vint_mb_Hubbard )

true

In [158]:
isapprox( Vint_mb , Vint_mb_einsum )

true

In [159]:
Vint_mb_einsum.data

330×330 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 36282 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣶⣶⣶⣦⣤⣤⣤⣤⣤⣤⣤⣄⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⠙⠦⡀⠻⢟⠳⣶⢤⣄⡈⠻⣟⠻⡿⣿⣛⠲⣶⢶⣦⢤⣤⣄⣀⡀⎥
⎢⣿⣿⣿⣿⣵⣿⣿⣿⣿⣿⣿⣿⣿⣯⣽⣶⣶⣾⣶⣶⣷⣾⣷⣯⣭⣤⣬⣅⣈⣈⣉⡁⠈⠉⠉⠉⠉⠉⠉⠉⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣙⣯⡙⢿⢿⣯⡙⢿⡻⣿⣗⢶⣶⣤⣄⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⢻⡦⣄⣿⣿⡷⢶⢤⣄⣸⣿⣿⣿⣿⣿⣿⡷⣶⢶⣶⡦⣤⣤⣀⡀⎥
⎢⢹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠈⣷⣌⢹⣯⣹⣿⣧⣍⡉⢻⢍⢿⣿⢭⣍⣻⣿⣷⣿⣿⣬⣭⣉⣉⎥
⎢⢸⣿⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠀⠘⣿⠀⢻⡏⢿⣿⣷⣯⠈⢷⡝⣿⢿⣿⡍⢿⣿⣿⡿⣿⣿⣾⣯⎥
⎢⠈⣿⣝⠛⢳⣿⣿⣿⣽⣛⡛⠛⠛⠛⢻⣶⣶⣾⣶⣶⣷⣾⣧⣯⣿⣤⣬⣇⣘⣈⣛⡃⠈⠋⠋⠛⠙⠛⠙⠛⎥
⎢⠀⣿⠈⠣⣸⣿⣿⣿⠈⢯⡙⢿⣶⣤⣸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠙⢯⡙⢟⢿⣭⡙⢿⣻⣿⣳⣶⣦⣤⣄⎥
⎢⠀⣿⣿⢆⢸⣿⣿⣿⣿⣿⡷⣶⣤⣀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡷⣶⢶⣶⢦⣤⣄⣀⡀⎥
⎢⠀⣿⢹⣦⣹⣿⣿⣿⢹⣏⣷⣾⣯⣍⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢿⣍⣿⣿⣭⣍⣷⣾⣿⣿⣯⣭⣭⣉⣉⎥
⎢⠀⢹⠀⢷⡽⣿⣿⣿⠀⢷⡍⢿⢿⣿⡭⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠈⢷⠽⣯⢿⣿⡍⢿⢿⣿⡽⣿⣿⣾⣯⎥
⎢⠀⠘⣦⡈⠃⣿⣟⢻⣶⣾⣧⣈⡋⠛⠛⣿⣟⠛⣿⣿⣿⣟⡛⠛⢻⣶⣾⣷⣿⣾⣿⣧⣬⣏⣛⣛⠙⠛⠙⠛⎥
⎢⠀⠀⣿⡙⠆⢿⣏⠻⣿⣿⣧⣕⣝⠷⠦⢿⣏⠳⣿⣿⣧⣽⣝⡗⢾⣿⣿⣿⣿⣿⣿⣧⣵⣽⣷⣮⣻⡿⠲⠶⎥
⎢⠀⠀⣿⣯⡂⢸⣿⣗⣿⣿⡟⣟⣿⣟⡒⢸⣿⣕⣿⣿⡟⣿⣯⣟⣻⣿⣿⣿⣿⣿⣿⡟⣟⣿⣟⣻⣿⣿⣛⣓⎥
⎢⠀⠀⢻⡘⠇⠸⣏⠻⢿⡿⣧⣹⡟⠿⠿⠸⣇⠻⢿⡿⢧⣽⡟⠿⠿⣿⠿⣿⣿⠿⠿⣧⣽⣟⣿⣿⠻⠿⠿⠿⎥
⎢⠀⠀⢸⣟⡆⠀⣿⡳⢸⣟⢿⣿⣿⣷⡦⠀⣿⣳⢸⣟⣾⣿⣿⣗⡦⢿⣕⣿⣿⣽⣷⢿⣿⣿⣿⣿⣿⣿⣲⡶⎥
⎢⠀⠀⠈⣟⡇⠀⢿⢿⠸⡿⣿⣿⣿⡿⣯⠀⢿⣻⠸⣟⡿⣿⣟⡿⣿⢸⡹⣿⣿⣹⣿⣿⣿⣿⣿⣿⣿⣿⢿⣿⎥
⎢⠀⠀⠀⢿⡇⠀⢸⣷⠀⣿⡆⣿⣿⣿⣷⠀⠸⣿⠀⢿⡇⣿⣿⣿⣷⠀⣿⡾⣿⣿⣿⡆⣿⣿⣿⣿⣿⣿⣿⣷⎥
⎣⠀⠀⠀⠸⡇⠀⠀⢿⠀⠸⡇⢸⡾⣿⣷⠀⠀⢿⠀⠸⡇⢸⡾⣿⣷⠀⢸⡆⢿⢸⣿⡇⢸⡾⣿⣷⢿⣿⣿⣿⎦

In [63]:
Vint_mb_Hubbard.data

6×6 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 6 stored entries:
 0.325738-1.00831e-17im           ⋅              …           ⋅    
          ⋅              0.303695+1.06068e-18im              ⋅    
          ⋅                       ⋅                          ⋅    
          ⋅                       ⋅                          ⋅    
          ⋅                       ⋅                          ⋅    
          ⋅                       ⋅              …  0.379433-1.90853e-17im

In [45]:
H_MB = H1_MB + Vint_mb_einsum
H_mb = H1_MB + Vint_mb_Hubbard

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)sparse([1, 2, 3, 2, 3, 4, 5, 1, 2, 3  …  6, 4, 5, 2, 4, 5, 6, 3, 5, 6], [1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 6], ComplexF64[-5.138363413839574 - 1.0e-17im, -1.6e-16 + 1.0e-17im, -3.1e-16 - 1.4e-16im, -5.160406455647614 - 2.0e-17im, 3.0e-17 + 5.0e-17im, -1.6e-16 + 1.0e-17im, -2.2e-16 - 1.0e-16im, -3.1e-16 + 1.6e-16im, -1.0e-17 + 7.0e-17im, -5.085939843890933 + 2.2e-16im  …  -3.1e-16 - 1.4e-16im, -5.047434948471091 - 4.0e-17im, 5.0e-17 + 7.0e-17im, -2.2e-16 + 1.1e-16im, -1.0e-17 + 1.0e-16im, -5.267796774627898 + 2.0e-16im, 5.0e-17 + 7.0e-17im, -3.1e-16 + 1.6e-16im, -1.0e-17 + 1.0e-16im, -5.084668254349429 + 4.3e-16im], 6, 6)

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [46]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [47]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-5.29743494847109, -5.297434948471089, -5.297434948471085, -4.964101615137757, -4.964101615137756, -4.9641016151377535]
[-5.2677967746278975, -5.1604064556476175, -5.138363413839573, -5.085939843890931, -5.084668254349429, -5.047434948471093]
[-5.340971176014529, -5.340971176014522, -5.340971176014509, -5.0661682656188365, -5.06616826561883, -5.066168265618821, -1.81285035447887, -1.8128503544788692, -1.8128503544788688, -1.704570336931173]

In [40]:
E_mb

6-element Vector{Float64}:
 -5.297434948471091
 -5.29743494847109
 -5.297434948471087
 -4.964101615137757
 -4.964101615137755
 -4.964101615137754